In [3]:
# Ouldridge Group Project 2024/25, Križan Jurinović
# Sequences generation and analysis script
 
# What it does:
# Generate mismatches systematically in a given string (oligostrand) sequence
# And analyse the properties of a given oligo / complex with the pfung and mfe function from the nupack library
 
from nupack import *
import itertools
import matplotlib.pyplot as plt
import pandas as pd
 
# Global parameter for the maximum number of mismatches
MAX_MISMATCHES = 3  # Adjustable
 
# Define strands
sX1 = Strand('GCGCCTAAATCACTCCTAACTCC', name='sX1')
sOi = Strand('CACTCCTAACTCCACCTCCATCCACCTAC', name = 'sOi')
 
domainvariant = "ATTTAGGCGC"  # Variant domain sequence, e.g. your variable toehold
sGi_original = Strand(f'GTGGAGTTAGGAGTG{domainvariant}', name='sGi_original')  # sGi including var domain
 
# Model definition
model1 = Model(material = 'dna', celsius = 25) #use standard settings, adjust to desired param, e.g. 25°C and salt conc
 
# Function to calculate detailed properties for a complex
def calculate_complex_properties(variant_strands, model, target_name):
    complex_set = ComplexSet(strands=variant_strands, complexes=SetSpec(max_size=2))
    results = complex_analysis(complexes=complex_set, model=model, compute=['pfunc', 'mfe'])
    for key in results.complexes.keys():
        if str(key) == target_name:
            c_result = results.complexes[key]
            return {
                'free_energy': c_result.free_energy,
                'partition_function': float(c_result.pfunc),
                'mfe_structure': c_result.mfe[0].structure if c_result.mfe else None,
                'mfe_structure_energy': c_result.mfe[0].energy if c_result.mfe else None
            }
    return None
 
# Function to generate mismatch seq
def generate_mismatches(sequence):
    bases = ['A', 'T', 'C', 'G']
    mismatch_variant = []
    seq_length = len(sequence)
   
    # Generate combinations using itertools lib, see documentary how this works in detail
    for num_mismatches in range(1, MAX_MISMATCHES + 1):
        for mismatch_positions in itertools.combinations(range(seq_length), num_mismatches):
            for mismatched_bases in itertools.product(bases, repeat=num_mismatches):
                sequence_list = list(sequence)
                mismatch_pos_record = []
                for pos, new_base in zip(mismatch_positions, mismatched_bases):
                    if sequence_list[pos] != new_base:  # Only introduce a mismatch
                        sequence_list[pos] = new_base
                        mismatch_pos_record.append(pos)
                mismatch_variant.append((''.join(sequence_list), mismatch_pos_record))
    return mismatch_variant
 
# Function to analyse mismatches
def analyse_mismatches(domain):
    mismatch_variant = generate_mismatches(domain)
    detailed_results = []
    detailed_results_Oi = []
   
    for variant, positions in mismatch_variant:
        sGi_variant = Strand(f'GTGGAGTTAGGAGTG{variant}', name=f'sGi_{variant}')
       
        # Calculate properties for sGi+sX1 complex
        sGi_sX1_name = f"<Complex ({sGi_variant.name}+sX1)>"
        complex_properties = calculate_complex_properties([sGi_variant, sX1], model1, sGi_sX1_name)
       
        if complex_properties:
            detailed_results.append({
                'Sequence': variant,
                'Mismatch Positions': [pos + 1 for pos in positions],  # Convert to 1-based index
                'Mismatch Count': len(positions),
                **complex_properties
            })

        # Calculate properties for SGi+SOi complex
        sGi_sOi_name = f"<Complex ({sGi_variant.name}+sOi)>"
        complex_properties_Oi = calculate_complex_properties([sGi_variant, sOi], model1, sGi_sOi_name)

        if complex_properties_Oi:
            detailed_results_Oi.append({
                **complex_properties_Oi
            })
             
    return [detailed_results, detailed_results_Oi]



 
# Run analysis
analysis_results = analyse_mismatches(domainvariant)
#print(analysis_results[1])
# Export results to CSV
def export_to_csv(results, filename):
    export_data = []
    resultcount = 0
    print(results[1])
    for result in results[0]:
        export_data.append({
            'Sequence': result['Sequence'],
            'Mismatch Positions': ', '.join(map(str, result['Mismatch Positions'])),  # Convert list to a comma-separated string
            'Mismatch Count': result['Mismatch Count'],
            'Free Energy (kcal/mol)': result['free_energy'],
            'Partition Function': result['partition_function'],
            'MFE Structure': result['mfe_structure'],
            'MFE Structure Energy (kcal/mol)': result['mfe_structure_energy'], 
            'Free Energy (kcal/mol)_Oi': results[1][resultcount]['free_energy'],
            'Partition Function_Oi': results[1][resultcount]['partition_function'],
            'MFE Structure_Oi': results[1][resultcount]['mfe_structure'],
            'DeltaG': result['free_energy'] - results[1][resultcount]['free_energy']
            #'MFE Structure Energy (kcal/mol)_Oi': results[1][resultcount]['mfe_structure_energy']
        })
        resultcount = resultcount + 1
            
    df = pd.DataFrame(export_data)
   # df.to_csv(filename, index=False)
    print(f"Results exported to {filename}")
 
# Save results to CSV
csv_filename = 'sGi_sX1_sOi_analysis_results.csv'
export_to_csv(analysis_results, csv_filename)
 
# Print results
#print("\n--- ~~~ results for variants ~~~ ---")
#for result in analysis_results[0]:
#    print(f"Sequence: {result['Sequence']}") # use sequence keyword in results to extract the sequence, do the same for the other variables
   # print(f"  Mismatch Positions: {result['Mismatch Positions']}")
  #  print(f"  Mismatch Count: {result['Mismatch Count']}")
   # print(f"  Free Energy: {result['free_energy']:.2f} kcal/mol")
  #  print(f"  Partition Function: {result['partition_function']:.2e}")
 #   print(f"  MFE Proxy Structure: {result['mfe_structure']}")
#    print(f"  Free Energy of MFE Structure: {result['mfe_structure_energy']:.2f} kcal/mol\n")
#for result in analysis_results[1]:
#    print(f"  Free Energy_Oi: {result['free_energy']:.2f} kcal/mol")
 
# Plot function in work, feel free to edit or create your own
 
# # Plot: Free Energy vs Mismatch Position for all mismatch counts
# plt.figure(figsize=(10, 6))
# for count in range(1, MAX_MISMATCHES + 1):
#     subset = [res for res in analysis_results if res['Mismatch Count'] == count]
#     if subset:
#         free_energies = [res['free_energy'] for res in subset]
#         mismatch_positions = [res['Mismatch Positions'][0] for res in subset]
#         plt.scatter(free_energies, mismatch_positions, label=f'{count} Mismatch{"es" if count > 1 else ""}', alpha=0.7)
 
# plt.xlabel('Free Energy (kcal/mol)')
# plt.ylabel('Mismatch Position (1-based index)')
# plt.title('Free Energy vs Mismatch Position')
# plt.legend()
# plt.grid(True)
# plt.show()

ModuleNotFoundError: No module named 'nupack'